In [2]:
import sqlite3
import pandas as pd

In [3]:
con = sqlite3.connect(':memory:')

In [4]:
table_names = [
    'Categories',   # категории товаров
    'Customers',    # заказчики
    'Employees',    # сотрудники
    'OrderDetails', # детали заказов
    'Orders',       # заказы
    'Products',     # товары
    'Shippers',     # перевозчики
    'Suppliers',    # поставщики
]

In [5]:
for table in table_names:
    data = pd.read_csv(f'{table}.csv', sep = ';')
    data.to_sql(table, con, index=False, if_exists='replace')

In [6]:
pd.read_sql(
    '''
    SELECT * FROM Categories
    ''',
    con,
)

,CategoryID,CategoryName,Description
0,1,Beverages,"Soft drinks, coffees, teas, beers, and ales"
1,2,Condiments,"Sweet and savory sauces, relishes, spreads, an..."
2,3,Confections,"Desserts, candies, and sweet breads"
3,4,Dairy Products,Cheeses
4,5,Grains/Cereals,"Breads, crackers, pasta, and cereal"
5,6,Meat/Poultry,Prepared meats
6,7,Produce,Dried fruit and bean curd
7,8,Seafood,Seaweed and fish


In [7]:
tasks = 0

### Все задания нужно выполнить только с помощью sql

1. Выберите заказчиков из Германии, Франции и Мадрида, выведите их название, страну и адрес.

In [8]:
tasks+=1
pd.read_sql(
    '''
    SELECT CustomerName, Country, Address 
    FROM 
        Customers
    WHERE Country IN ('Germany', 'France', 'Spain')
    ''',
    con,
)

,CustomerName,Country,Address
0,Alfreds Futterkiste,Germany,Obere Str. 57
1,Blauer See Delikatessen,Germany,Forsterstr. 57
2,Blondel père et fils,France,"24, place Kléber"
3,Bólido Comidas preparadas,Spain,"C/ Araquil, 67"
4,Bon app',France,"12, rue des Bouchers"
5,Drachenblut Delikatessend,Germany,Walserweg 21
6,Du monde entier,France,"67, rue des Cinquante Otages"
7,FISSA Fabrica Inter. Salchichas S.A.,Spain,"C/ Moralzarzal, 86"
8,Folies gourmandes,France,"184, chaussée de Tournai"
9,Frankenversand,Germany,Berliner Platz 43


2. Выберите топ 3 страны по количеству заказчиков, выведите их названия и количество записей.

In [9]:
tasks+=1
pd.read_sql(
    '''
    SELECT Country, count(*) as cnt 
    FROM 
        Customers
    GROUP BY Country
    ORDER BY count(*) desc
    LIMIT 3
    ''',
    con,
)

,Country,cnt
0,USA,13
1,Germany,11
2,France,11


3. Выберите перевозчика, который отправил 10-й по времени заказ, выведите его название, и дату отправления.

In [10]:
tasks+=1
pd.read_sql(
    '''
    SELECT ShipperName, OrderDate 
    FROM (
        SELECT *
        , row_number() over (
        order by substr(OrderDate, 7) || "-" || substr(OrderDate,4,2)  || "-" || substr(OrderDate, 1,2) asc
        ) as rn 
        FROM 
            Orders o
        INNER JOIN 
            Shippers s 
        ON 
            o.ShipperID=s.ShipperID)
    WHERE rn = 10
    ''',
    con,
)

,ShipperName,OrderDate
0,Federal Shipping,16.07.1996


4. Выберите самый дорогой заказ, выведите список товаров с их ценами.

In [11]:
tasks+=1
pd.read_sql(
    '''
    SELECT p.ProductID, p.ProductName, p.Price
    FROM OrderDetails od
    INNER JOIN 
        Products p
    ON
        p.ProductID=od.ProductID
    WHERE OrderID = (
        SELECT OrderID
        FROM Products p
        INNER JOIN OrderDetails o ON p.ProductID=o.ProductID
        GROUP BY OrderID
        ORDER BY sum(Price*Quantity) desc
        LIMIT 1)
    ''',
    con,
)

,ProductID,ProductName,Price
0,20,Sir Rodney's Marmalade,81
1,38,Côte de Blaye,263.5
2,60,Camembert Pierrot,34
3,72,Mozzarella di Giovanni,34.8


5. Какой товар больше всего заказывали по количеству единиц товара, выведите его название и количество единиц в каждом из заказов.

In [13]:
tasks+=1
pd.read_sql(
    '''
    SELECT p.ProductName, o.Quantity, o.OrderID
    FROM 
        Products p
    INNER JOIN 
        OrderDetails o 
    ON 
        p.ProductID=o.ProductID
    INNER JOIN
        (SELECT ProductID
        FROM 
            OrderDetails
        GROUP BY ProductID
        ORDER BY sum(Quantity) desc
        LIMIT 1) f
    ON
        p.ProductID=f.ProductID
    ''',
    con,
)

,ProductName,Quantity,OrderID
0,Gorgonzola Telino,20,10253
1,Gorgonzola Telino,40,10272
2,Gorgonzola Telino,15,10273
3,Gorgonzola Telino,4,10325
4,Gorgonzola Telino,25,10335
5,Gorgonzola Telino,56,10342
6,Gorgonzola Telino,30,10356
7,Gorgonzola Telino,70,10359
8,Gorgonzola Telino,20,10363
9,Gorgonzola Telino,30,10374


6. Выведите топ 5 поставщиков по количеству заказов, выведите их названия, страну, контактное лицо и телефон.

In [14]:
tasks+=1
pd.read_sql(
    '''
    SELECT s.SupplierName, s.Country, s.ContactName, s.Phone
    FROM 
        OrderDetails o
    INNER JOIN
        Products p
    ON
        o.ProductID=p.ProductID
    LEFT JOIN
        Suppliers s
    ON
        p.SupplierID=s.SupplierID
    GROUP BY p.SupplierID
    ORDER BY count(distinct OrderID) desc
    LIMIT 5
    ''',
    con,
)

,SupplierName,Country,ContactName,Phone
0,Plutzer Lebensmittelgroßmärkte AG,Germany,Martin Bein,(069) 992755
1,"Pavlova, Ltd.",Australia,Ian Devling,(03) 444-2343
2,"Specialty Biscuits, Ltd.",UK,Peter Wilson,(161) 555-4448
3,Norske Meierier,Norway,Beate Vileid,(0)2-953010
4,Formaggi Fortini s.r.l.,Italy,Elio Rossi,(0544) 60323


7. Какую категорию товаров заказывали больше всего по стоимости в Бразилии, выведите страну, название категории и сумму.

In [15]:
tasks+=1
pd.read_sql(
    '''
    SELECT cu.Country, c.CategoryName, sum(Quantity*Price) as sum
    FROM 
        OrderDetails od
    INNER JOIN
        Products p
    ON
        od.ProductID=p.ProductID
    INNER JOIN
        Categories c
    ON
        p.CategoryID=c.CategoryID
    INNER JOIN
        Orders o
    ON
        od.OrderID=o.OrderID
    INNER JOIN
        Customers cu
    ON
        o.CustomerID=cu.CustomerID and cu.Country='Brazil'
    GROUP BY c.CategoryID
    ORDER BY sum(Quantity*Price) desc
    LIMIT 1
    ''',
    con,
)

,Country,CategoryName,sum
0,Brazil,Beverages,13636.0


8. Какая разница в стоимости между самым дорогим и самым дешевым заказом из США.

In [16]:
tasks+=1
pd.read_sql(
    '''
    SELECT max(sum) - min(sum) as diff 
        FROM(
        SELECT cu.Country
        , od.OrderID
        , sum(Quantity*Price) as sum
        FROM 
            OrderDetails od
        INNER JOIN
            Products p
        ON
            od.ProductID=p.ProductID
        INNER JOIN
            Categories c
        ON
            p.CategoryID=c.CategoryID
        INNER JOIN
            Orders o
        ON
            od.OrderID=o.OrderID
        INNER JOIN
            Customers cu
        ON
            o.CustomerID=cu.CustomerID and cu.Country='USA'
        GROUP BY od.OrderID)
    ''',
    con,
)

,diff
0,7698.45


In [17]:
pd.read_sql(
    '''
        SELECT *
        FROM 
            OrderDetails od
                INNER JOIN
            Products p
        ON
            od.ProductID=p.ProductID
        WHERE OrderID=10349
    ''',
    con,
)

,OrderDetailID,OrderID,ProductID,Quantity,ProductID,ProductName,SupplierID,CategoryID,Unit,Price
0,272,10349,54,24,54,Tourtière,25,6,16 pies,июль.45


9. Выведите количество заказов у каждого их трех самых молодых сотрудников, а также имя и фамилию во второй колонке.

In [18]:
tasks+=1
pd.read_sql(
    '''
    SELECT 
    count(distinct OrderID) as cnt_orders,
    FirstName ||' '|| LastName as Emp,
    DATE('now') -
    substr(BirthDate, 7) || "-" || substr(BirthDate,4,2)  || "-" || substr(BirthDate, 1,2) as Age,
    substr(BirthDate, 7) || "-" || substr(BirthDate,4,2)  || "-" || substr(BirthDate, 1,2) as NewBirth
    FROM Employees e
    INNER JOIN
        Orders o
    ON
        e.EmployeeID=o.EmployeeID
    GROUP BY e.EmployeeID
    ORDER BY substr(BirthDate, 7) || "-" || substr(BirthDate,4,2)  || "-" || substr(BirthDate, 1,2) desc
    LIMIT 3
    ''',
    con,
)

,cnt_orders,Emp,Age,NewBirth
0,6,Anne Dodsworth,54,1969-07-02
1,29,Nancy Davolio,55,1968-12-08
2,31,Janet Leverling,60,1963-08-30


10. Сколько банок крабового мяса всего было заказано.

In [19]:
tasks+=1
pd.read_sql(
    '''
    SELECT sum(Quantity) as crab_meat
    FROM 
        Products p
    INNER JOIN
        OrderDetails od
    ON
        p.ProductID=od.ProductID
    WHERE lower(p.ProductName) like '%crab%'
    ''',
    con,
)

,crab_meat
0,256


In [20]:
if tasks==10:
    print('Выполнены все задания 🙂')
else:
    print('Выполнены не все задания! 🙀')

Выполнены все задания 🙂
